In [5]:
!pip install biopython

In [6]:
!pip install tensorflow

In [7]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from Bio.SeqIO import parse
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from sklearn.preprocessing import LabelEncoder
import re
import math
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv1D, MaxPooling1D
from collections import Counter
from sklearn.utils import class_weight
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [8]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()
df = pd.read_csv(io.StringIO(uploaded['final_dates_reduced.csv'].decode('utf-8')))
df.head(2)

Saving final_dates_reduced.csv to final_dates_reduced (1).csv


,Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date,Sequence
0,MT683386,2020-07-01T00:00:00Z,Severe acute respiratory syndrome-related coro...,29858,USA,Homo sapiens,NaN,Apr-May,GGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTT...
1,MT683387,2020-07-01T00:00:00Z,Severe acute respiratory syndrome-related coro...,29854,USA,Homo sapiens,NaN,Apr-May,ATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGA...


In [9]:
!pip install keras-metrics

In [10]:
# df3 = df.sample(frac=1).reset_index(drop=True)
# df3.head(5)

In [11]:
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

In [12]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [13]:
def ordinal_encoder(my_array):
    
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    
    return float_encoded

In [14]:
data=df[["Sequence","Collection_Date"]]
data=data[data["Sequence"].notna()]

dummy=[]
dum=np.array(dummy)

form={"inp_seq":dum}

seq_df = pd.DataFrame (form, columns = ['inp_seq'])

seq_list=[]

for idx, seq in enumerate(list(data["Sequence"])):
    arr=ordinal_encoder(string_to_array(seq))
    seq_list.append(arr)
    
seq_df["inp_seq"]=seq_list

final_data= data.assign(enc_seq=seq_df)

final_data=final_data[["enc_seq","Collection_Date"]]

final_data

,enc_seq,Collection_Date
0,"[0.75, 0.75, 1.0, 1.0, 1.0, 0.25, 1.0, 0.25, 0...",Apr-May
1,"[0.25, 1.0, 0.25, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5...",Apr-May
2,"[0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, 0.25, ...",Apr-May
3,"[0.75, 0.75, 1.0, 0.25, 0.25, 0.5, 0.25, 0.25,...",Apr-May
4,"[0.75, 0.5, 1.0, 1.0, 0.25, 0.5, 0.75, 0.75, 1...",Apr-May
...,...,...
5592,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Jan-Feb
5593,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Jan-Feb
5594,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Jan-Feb
5595,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Jan-Feb


In [15]:
def get_maxLen(enc_seq):
    
    max=0
    for row in enc_seq:
        #print(type(row))
        if(len(row)>max):
            max=len(row)
    
    return max

In [16]:
def append_arr(enc_seq, max_len):
    
    seq_l=list(enc_seq)
    for i in range(len(seq_l),max_len):
        seq_l.append(0)
        
    new_seq_ar=np.array(seq_l)

        
    return new_seq_ar

In [17]:
import math

max_len=get_maxLen(final_data["enc_seq"])
# print("max_len is",max_len)
# if max_len%2 !=0:
#     max_len += 1
max_len = 29929

n = 1
for i in range(1, math.floor(math.sqrt(max_len))):
    if max_len%i==0:
        n = i

padded_seq_list=[]

for index, row in final_data.iterrows():
    seq_ar=append_arr(row["enc_seq"],max_len)
    padded_seq_list.append(seq_ar)

dummy=[]
dum=np.array(dummy)
form={"padded_enc_seq":dum}
padded_seq_df = pd.DataFrame (form, columns = ['padded_enc_seq'])

padded_seq_df["padded_enc_seq"]=padded_seq_list

padded_final_data= final_data.assign(padded_enc_seq=padded_seq_df)

padded_final_data=padded_final_data[["padded_enc_seq","Collection_Date"]]

padded_final_data.head(5)

,padded_enc_seq,Collection_Date
0,"[0.75, 0.75, 1.0, 1.0, 1.0, 0.25, 1.0, 0.25, 0...",Apr-May
1,"[0.25, 1.0, 0.25, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5...",Apr-May
2,"[0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, 0.25, ...",Apr-May
3,"[0.75, 0.75, 1.0, 0.25, 0.25, 0.5, 0.25, 0.25,...",Apr-May
4,"[0.75, 0.5, 1.0, 1.0, 0.25, 0.5, 0.75, 0.75, 1...",Apr-May


In [18]:
def reshape_seq(seq, m ,n):
    
    r_seq=np.reshape(seq,(m,n))
    
    return r_seq

re_seqList=[]

for index, row in padded_final_data.iterrows():
    #seq_ar=reshape_seq(row["padded_enc_seq"],n,max_len//n)
    seq_ar=reshape_seq(row["padded_enc_seq"],173, 173)
    re_seqList.append(seq_ar)

In [19]:
X = np.asarray(re_seqList)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))
X.shape

(5597, 173, 173, 1)

In [20]:
Y = padded_final_data['Collection_Date'].values
Y.shape

(5597,)

In [21]:
loc_classes = list(Y)
loc_classes = np.array(loc_classes) 
loc_classes = list(np.unique(loc_classes))

In [22]:
for i, x in enumerate(loc_classes):
    print(i, x)

0 Apr-May
1 Jan-Feb
2 Mar


In [23]:
df2 = padded_final_data['Collection_Date'].apply(loc_classes.index)
Y = df2.values
Y.shape

(5597,)

In [24]:
#x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.30)

In [25]:
model_best = Sequential()

In [26]:
class CheckMetrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_accs = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, Y_val = self.validation_data[0],self.validation_data[1]
                
        Y_val = np.argmax(Y_val,axis=1)
        
        Y_pred = self.model.predict(X_val)
        Y_pred = np.argmax(Y_pred,axis=1)

        _val_acc = accuracy_score(Y_val, Y_pred)

        self.val_accs.append(_val_acc)
        
        if _val_acc == max(self.val_accs):
            print("Validation Accuracy has improved. Saving Model.")
            self.model.save('modelCNN.h5')
            model_best = self.model

        return

In [27]:
keras_callbacks = CheckMetrics()

K-Fold Cross Validation

In [28]:
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=100)
cvscores = []
k = 0
for train, test in kfold.split(X, Y):
    k += 1
    batch_size = 32 
    num_classes = 3
    epochs = 29
    img_rows, img_cols = X[train].shape[1], X[train].shape[2]
    # X_train = X[train].reshape(X[train].shape[0],img_rows,img_cols,1)
    # X_test = X[test].reshape(X[test].shape[0],img_rows,img_cols,1)
    # convert class vectors to binary class matrices
    Y_train = keras.utils.to_categorical(Y[train], num_classes)
    Y_test = keras.utils.to_categorical(Y[test], num_classes)

    # create model
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_rows, img_cols, 1)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
  
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(3, 3)))
  
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Conv2D(128, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))
  
    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

    # Fit the model
    history1 = model.fit(X[train], Y_train, batch_size=batch_size, epochs=200, verbose=1, validation_data=(X[test], Y_test), callbacks=[keras_callbacks])
 
    # Evaluate the model
    model_sel = load_model('modelCNN.h5')
    scores = model_sel.evaluate(X[test], Y_test, verbose=0)
    print("%s: %.2f%%" % (model_sel.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

    # Confusion matrix
    Y_pred = model_sel.predict(X[test])
    y_pred = np.argmax(Y_pred, axis=1)

    print('Confusion Matrix - Test')
    y_val2 = np.argmax(Y_test, axis=1)

    print(confusion_matrix(y_val2, y_pred))
    print('Classification Report')
    target_names = ['Apr-May', 'Jan-Feb', 'Mar']
    print(classification_report(y_val2, y_pred, target_names=target_names))
    print("---------------------------------------------------------------------------------------------------\n\n")

Train on 3731 samples, validate on 1866 samples
Epoch 1/200
3731/3731 [==============================] - 35s 9ms/step - loss: 0.9242 - accuracy: 0.6065 - val_loss: 0.8605 - val_accuracy: 0.5504
Validation Accuracy has improved. Saving Model.
Epoch 2/200
3731/3731 [==============================] - 26s 7ms/step - loss: 0.7570 - accuracy: 0.6309 - val_loss: 0.9507 - val_accuracy: 0.5504
Validation Accuracy has improved. Saving Model.
Epoch 3/200
3731/3731 [==============================] - 26s 7ms/step - loss: 0.7141 - accuracy: 0.6457 - val_loss: 0.9648 - val_accuracy: 0.5563
Validation Accuracy has improved. Saving Model.
Epoch 4/200
3731/3731 [==============================] - 26s 7ms/step - loss: 0.6964 - accuracy: 0.6583 - val_loss: 0.7482 - val_accuracy: 0.5729
Validation Accuracy has improved. Saving Model.
Epoch 5/200
3731/3731 [==============================] - 26s 7ms/step - loss: 0.6754 - accuracy: 0.6626 - val_loss: 0.6743 - val_accuracy: 0.6442
Validation Accuracy has improv

In [1]:
# for i in range(0, len(cvscores)):
#     print(f'> Fold {i+1}: Validation Accuracy = {cvscores[i]}%')

# print(f'\nAverage Validation Accuracy: {np.mean(cvscores)}')
# print(f'Standard Deviation: {np.std(cvscores)}')